In [ ]:
# print(full_text)

In [ ]:
# imaged

In [22]:
#   --disable_image_extraction      Disable image extraction.
#   --paginate_output               Paginate output.
#   --disable_multiprocessing       Disable multiprocessing.
#   --languages TEXT                Comma separated list of languages to use for
#                                   OCR.
#   --config_json TEXT              Path to JSON file with additional
#                                   configuration.
#   --processors TEXT               Comma separated list of processors to use.
#                                   Must use full module path.
#   --force_ocr                     Force OCR on the whole document.
#   --page_range TEXT               Page range to convert, specify comma
#                                   separated page numbers or ranges.  Example:
#                                   0,5-10,20
#   --output_format [markdown|json|html]
#                                   Format to output results in.
#   -d, --debug                     Enable debug mode.
#   --output_dir PATH               Directory to save output.
#   --chunk_idx INTEGER             Chunk index to convert
#   --num_chunks INTEGER            Number of chunks being processed in parallel
#   --max_files INTEGER             Maximum number of pdfs to convert
#   --workers INTEGER               Number of worker processes to use.
#   --skip_existing                 Skip existing converted files.
#   --help                          Show this message and exit.

example of code to be adjusted, then push the chunks with metadata to Qdrant

In [23]:
# from marker.converters.pdf import PdfConverter
# from marker.models import create_model_dict
# from marker.config.parser import ConfigParser
# from marker.output import text_from_rendered


# config = {
#     "output_format": "markdown",
#     "paginate_output": True
# }
# config_parser = ConfigParser(config)

# converter = PdfConverter(
#     config=config_parser.generate_config_dict(),
#     artifact_dict=create_model_dict(),
#     processor_list=config_parser.get_processors(),
#     renderer=config_parser.get_renderer()
# )
# rendered = converter("C:\\Users\\fiori\\OneDrive\\Documenti\\Rulebooks\\SpellBook_391834.pdf")
# text, meta, images = text_from_rendered(rendered)

Loaded layout model datalab-to/surya_layout0 on device cuda with dtype torch.float16
Loaded texify model to cuda with torch.float16 dtype
Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16
Loaded table recognition model vikp/surya_tablerec on device cuda with dtype torch.float16
Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16


Recognizing layout: 100%|██████████| 3/3 [00:34<00:00, 11.41s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 0it [00:00, ?it/s]
Recognizing tables: 0it [00:00, ?it/s]


# INGEST DATA

In [81]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered
from langchain.text_splitter import MarkdownHeaderTextSplitter
import fitz  # PyMuPDF
import os

In [95]:
def read_token_from_file(file_path="token.txt"):
    with open(file_path, "r") as file:
        return file.read().strip()

In [97]:
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain.embeddings import OpenAIEmbeddings  # Correct import path for OpenAI embeddings
import os

COLLECTION_NAME = "automatic_ingestion"
os.environ["OPENAI_API_KEY"] = read_token_from_file("keys\OpenAI.txt")
URL=read_token_from_file("keys\qdrant_URL.txt")
API_KEY=read_token_from_file("keys\qdrant.txt")
# Specify the folder path
# folder_path = "C:\\Users\\fiori\\OneDrive\\Documenti\\Rulebooks"
# EMBEDDING_MODEL_NAME = "thenlper/gte-small"

In [78]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

# 1. Initialize MarkdownHeaderTextSplitter with the level of header to split on.
# text_splitter = MarkdownHeaderTextSplitter(levels=["1"])
headers_to_split = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
    ("#####", "Header 5"),
    ("######", "Header 6"),
]


text_splitter = MarkdownHeaderTextSplitter(
    headers_to_split,
    strip_headers=False
)

In [79]:
def find_headers_in_pdf(pdf_path, headers):
    """
    Finds specified headers in a PDF and returns their page numbers.

    :param pdf_path: Path to the PDF file
    :param headers: Dictionary of headers to search for
    :return: Dictionary mapping headers to lists of page numbers
    """
    # Open the PDF file
    document = fitz.open(pdf_path)
    header_pages = {header: [] for header in headers.values()}
    
    # Iterate through each page
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text = page.get_text("text")  # Extract raw text
        
        # Split text into lines for line-by-line matching
        lines = text.split("\n")
        
        for line in lines:
            for _, header_value in headers.items():
                # Match exact line content with header
                if line.strip() == header_value.strip()[2:-2]:
                    header_pages[header_value].append(page_num + 1)  # 1-indexed
    
    document.close()
    return header_pages

In [80]:
converter = PdfConverter(
    artifact_dict=create_model_dict(),
)

Loaded layout model datalab-to/surya_layout0 on device cuda with dtype torch.float16
Loaded texify model to cuda with torch.float16 dtype
Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16
Loaded table recognition model vikp/surya_tablerec on device cuda with dtype torch.float16
Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16


In [93]:
FOLDER = "C:\\Users\\fiori\\OneDrive\\Documenti\\Rulebooks"
to_ingest = []

# Get all file names in the folder
file_names = os.listdir(FOLDER)
for file_name in file_names:
    rendered = converter(FOLDER + "\\"+ file_name)


    text, _, images = text_from_rendered(rendered)
    chunks = text_splitter.split_text(text)


    #ADD metadata pages
    result = []
    for chunk in chunks:
        result.append(find_headers_in_pdf(FOLDER + "\\"+ file_name, chunk.metadata))

    pages = []  # pages for each chunk
    for el in result:
        values = list(el.values())  # Convert values to a list
        
        # Check if values has content
        if values:
            # Ensure the first element is not empty
            if values[0]:
                if len(values) >= 2 and values[0] != values[-1] and values[-1]:
                    pages.append(f"{values[0][0]} - {values[-1][0]}")
                else:
                    pages.append(f"{values[0][0]}")
            else:
                pages.append("Not found in Rulebook")
        else:
            pages.append("Not found in Rulebook")

    txt_dict = {
            "game_name": file_name.split("_")[0],
            "game_id": file_name.split("_")[1][:-4]#remove .pdf
        }

    # Add txt_dict to the metadata of each chunk
    for i, chunk in enumerate(chunks):
        chunk.metadata['game_name'] = txt_dict['game_name']
        chunk.metadata['game_id'] = txt_dict['game_id']
        chunk.metadata['pages'] = pages[i]
        to_ingest.append(chunk)



Recognizing layout: 100%|██████████| 3/3 [00:32<00:00, 10.94s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 0it [00:00, ?it/s]
Recognizing layout: 100%|██████████| 3/3 [00:29<00:00,  9.84s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 0it [00:00, ?it/s]
Recognizing tables: 0it [00:00, ?it/s]
Recognizing layout: 100%|██████████| 1/1 [00:08<00:00,  8.73s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 0it [00:00, ?it/s]
Recognizing tables: 0it [00:00, ?it/s]
Recognizing layout: 100%|██████████| 2/2 [00:14<00:00,  7.50s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 0it [00:00, ?it/s]
Recognizing tables: 0it [00:00, ?it/s]


In [98]:
def add_embeddings_to_qdrant(docs_processed, URL, API_KEY, OPENAI_KEY, collection_name="test_2"):
    """Create embeddings for documents and add them to Qdrant."""

    URL=URL
    API_KEY=API_KEY

    if OPENAI_KEY:
        embedding_model = OpenAIEmbeddings(
            model="text-embedding-ada-002",  # Specify the desired OpenAI embedding model
        )

    vector_store = QdrantVectorStore.from_documents(
        docs_processed,
        embedding_model,
        url = URL,
        #prefer_grpc=True,
        api_key=API_KEY,
        collection_name=collection_name,
        force_recreate = False
    )

In [100]:
add_embeddings_to_qdrant(to_ingest, URL, API_KEY, os.environ["OPENAI_API_KEY"], COLLECTION_NAME)

QDRANT

# TESTS

In [28]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered

converter = PdfConverter(
    artifact_dict=create_model_dict(),
)
rendered = converter("C:\\Users\\fiori\\OneDrive\\Documenti\\Rulebooks\\SpellBook_391834.pdf")
text, _, images = text_from_rendered(rendered)

Loaded layout model datalab-to/surya_layout0 on device cuda with dtype torch.float16
Loaded texify model to cuda with torch.float16 dtype
Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16
Loaded table recognition model vikp/surya_tablerec on device cuda with dtype torch.float16
Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16


Recognizing layout: 100%|██████████| 3/3 [00:27<00:00,  9.20s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 0it [00:00, ?it/s]
Recognizing tables: 0it [00:00, ?it/s]


levare noise dai chunks


Blip fa un po' cagare, andrebbe cambiato in meglio

In [29]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

# 1. Initialize MarkdownHeaderTextSplitter with the level of header to split on.
# text_splitter = MarkdownHeaderTextSplitter(levels=["1"])
headers_to_split = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
    ("#####", "Header 5"),
    ("######", "Header 6"),
]


text_splitter = MarkdownHeaderTextSplitter(
    headers_to_split,
    strip_headers=False
)

chunks = text_splitter.split_text(text)

get page of headers

In [56]:
import fitz  # PyMuPDF

def find_headers_in_pdf(pdf_path, headers):
    """
    Finds specified headers in a PDF and returns their page numbers.

    :param pdf_path: Path to the PDF file
    :param headers: Dictionary of headers to search for
    :return: Dictionary mapping headers to lists of page numbers
    """
    # Open the PDF file
    document = fitz.open(pdf_path)
    header_pages = {header: [] for header in headers.values()}
    
    # Iterate through each page
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text = page.get_text("text")  # Extract raw text
        
        # Split text into lines for line-by-line matching
        lines = text.split("\n")
        
        for line in lines:
            for _, header_value in headers.items():
                # Match exact line content with header
                if line.strip() == header_value.strip()[2:-2]:
                    header_pages[header_value].append(page_num + 1)  # 1-indexed
    
    document.close()
    return header_pages

pdf_path = "C:\\Users\\fiori\\OneDrive\\Documenti\\Rulebooks\\SpellBook_391834.pdf"  # Replace with your PDF file path

# Input PDF file path and headers
headers = {"2":"**wizard**"}

# Find headers
header_pages = find_headers_in_pdf(pdf_path, headers)

# Print results
for header, pages in header_pages.items():
    print(f"'{header}' found on page(s): {', '.join(map(str, pages)) if pages else 'Not found'}")


'**wizard**' found on page(s): Not found


In [71]:
#ADD metadata pages
result = []
for chunk in chunks:
    result.append(find_headers_in_pdf(pdf_path, chunk.metadata))

pages = [] #pages for each chunk
for el in result:
    values = list(el.values())  # Convert values to a list
    l = len(values)  # Get the length of the list
    if l >= 1:
        if l >= 2 and values[0] != values[-1] and (values[-1] and values[0]):
            pages.append(f"{values[0][0]} - {values[-1][0]}")
        else:
            pages.append(f"{values[0][0]}")
    else:
        pages.append("Not found in Rulebook")
        

book_name = "SpellBook_391834"
txt_dict = {
        "game_name": book_name.split("_")[0],
        "game_id": book_name.split("_")[1]
}
# Add txt_dict to the metadata of each chunk
for i, chunk in enumerate(chunks):
    chunk.metadata['game_name'] = txt_dict['game_name']
    chunk.metadata['game_id'] = txt_dict['game_id']
    chunk.metadata['pages'] = pages[i]

In [72]:
len(pages) == len(chunks)

True

In [73]:
#add metadata
# for txt_file in tqdm(txt_files):
#     # Read text content from each TXT file
#     with open(os.path.join(path, txt_file), 'r', encoding='utf-8') as file:
#         text = file.read()
#     txt_texts.append(text)

# Read text content from each TXT file
# txt_dict = [{"game_name": txt_file.removesuffix(".txt").split("_")[0], "game_id": txt_file.removesuffix(".txt").split("_")[1]} for txt_file in txt_files]

book_name = "SpellBook_391834"
txt_dict = {
        "game_name": book_name.split("_")[0],
        "game_id": book_name.split("_")[1]
    }
    


In [74]:

# Add txt_dict to the metadata of each chunk
for i, chunk in enumerate(chunks):
    chunk.metadata['game_name'] = txt_dict['game_name']
    chunk.metadata['game_id'] = txt_dict['game_id']
    chunk.metadata['pages'] = pages[i]

In [76]:
chunks[2].metadata

{'Header 1': '**CONTENTS**',
 'Header 2': '**Spell Cards**',
 'game_name': 'SpellBook',
 'game_id': '391834',
 'pages': '2'}

In [87]:
def read_token_from_file(file_path="token.txt"):
    with open(file_path, "r") as file:
        return file.read().strip()

In [88]:
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain.embeddings import OpenAIEmbeddings  # Correct import path for OpenAI embeddings
import os

COLLECTION_NAME = "automatic_ingestion"
os.environ["OPENAI_API_KEY"] = read_token_from_file("/content/OpenAI.txt")
URL=read_token_from_file("/content/qdrant_URL.txt")
API_KEY=read_token_from_file("/content/qdrant.txt")
# Specify the folder path
# folder_path = "C:\\Users\\fiori\\OneDrive\\Documenti\\Rulebooks"
# EMBEDDING_MODEL_NAME = "thenlper/gte-small"

In [89]:
def add_embeddings_to_qdrant(docs_processed, URL, API_KEY, OPENAI_KEY, collection_name="test_2"):
    """Create embeddings for documents and add them to Qdrant."""

    URL=URL
    API_KEY=API_KEY

    if OPENAI_KEY:
        embedding_model = OpenAIEmbeddings(
            model="text-embedding-ada-002",  # Specify the desired OpenAI embedding model
        )


    vector_store = QdrantVectorStore.from_documents(
        docs_processed,
        embedding_model,
        url = URL,
        #prefer_grpc=True,
        api_key=API_KEY,
        collection_name=collection_name,
        force_recreate = False
    )

In [90]:
add_embeddings_to_qdrant(chunks, URL, API_KEY, os.environ["OPENAI_API_KEY"], COLLECTION_NAME)

<ipython-input-89-80bcea435d12>:8: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(
